# Opciones

Una opción es un contrato que da al comprador, el derecho de entrar en otro contrato durante un periodo de tiempo en particular. Este contrato recibe el nombre de **derivado** ya que su valor depende del valor de otro contrato, el **subyacente**. Este contrato subyacente puede ser una acción, un bono o algún activo financiero como un commodity, una moneda o una tasa de interés.

Si la opción da el derecho de comprar el subyacente, se dice que es una **opción call**, en cambio, si se da el derecho de venderlo, se habla de una **opción put**.

En este notebook consideraremos únicamente aquellas opciones que sólo tienen una fecha de ejercicio, es decir, **opciones europeas**.

## Fórmula de Black-Scholes-Merton
Para valuar el precio de una opción call europea con subyacente $S$ y precio de ejercico (strike) $K$, el modelo de Black-Scholes-Merton establace la siguiente fórmula:

$$
\begin{array}{rll}
 Call(S_{t}, K, t, T, r, \sigma) & = & S_{t}N(d_1) - e^{-r(T-t)} K N(d_2) \\
 N(d) & = &\dfrac{1}{\sqrt{2\pi}} \int_{- \infty}^{d} e^{- 0.5 x^2}dx \\
d_1 & = & \dfrac{ Ln \Big(\dfrac{S_t}{K}\Big) + \Big( r + \dfrac{\sigma^2}{2} \Big) \Big(T - t \Big) }{ \sigma \sqrt{T-t} } \\
d_2 & = & \dfrac{ Ln \Big(\dfrac{S_t}{K}\Big) - \Big( r + \dfrac{\sigma^2}{2} \Big) \Big(T - t \Big) }{ \sigma \sqrt{T-t} } 
\end{array}
$$

en donde $r$ es la tasa libre de riesgo, $T$ es la fecha de vencimiento de la opción, $t$ es la fecha de valuación y $N$ es la función de distribución de una variable aleatoria normal estándar.

### Ejercicio

+ Implemente la ecuación de Black-Scholes-Merton para valuar una opción call europea y prueba su función con los siguientes datos:
    * $S_{t} = 100$
    * $K = 97$
    * $\sigma = 0.2$
    * $r = 0.01$
    * $T = 2020/08/18$
    * $t = 2019/08/18$

**Sugerencia** Para calcular la función $N(d)$, puede utilizar el método *cdf* de la clase norm
```python
from scipy.stats import norm
help(norm.cdf)
```
    

In [1]:
#Escriba aquí su código
import numpy as np
from scipy.stats import norm